In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ast

In [18]:
# Load the datasets
predicted_tape_path = '../predicted_tape.csv'
real_tape_path = '../Tapes/UoB_Set01_2025-01-02tapes.csv'
lob_dataset_path = '../lob.csv'

predicted_tape = pd.read_csv(predicted_tape_path)
real_tape = pd.read_csv(real_tape_path)
lob_dataset = pd.read_csv(lob_dataset_path)

# Display the first few rows of each dataset to understand their structure
predicted_tape.head(), real_tape.head(), lob_dataset.head()


(             timestamp  transaction_price
 0  2025-01-02 00:00:10              267.0
 1  2025-01-02 00:00:11              268.0
 2  2025-01-02 00:00:12              270.0
 3  2025-01-02 00:00:13              267.0
 4  2025-01-02 00:00:14              267.0,
    10.881   267   1
 0  11.067   269   1
 1  11.222   267   2
 2  12.338   270   2
 3  13.733   267   3
 4  18.321   265   2,
    Unnamed: 0  Timestamp       Bid         Ask
 0           0      0.000        []          []
 1           1      0.279  [[1, 6]]          []
 2           2      1.333  [[1, 6]]  [[800, 1]]
 3           3      1.581  [[1, 6]]  [[799, 1]]
 4           4      1.643  [[1, 6]]  [[798, 1]])

In [19]:
# Assign column names to the Real Tape dataset for clarity
real_tape.columns = ['seconds_from_midnight', 'price', 'quantity']

# Convert the 'seconds_from_midnight' to a datetime format, using the base date of 2025-01-02
base_date = pd.Timestamp('2025-01-02')
real_tape['timestamp'] = real_tape['seconds_from_midnight'].apply(lambda x: base_date + pd.Timedelta(seconds=x))

# Drop the original 'seconds_from_midnight' column as it's no longer needed
real_tape.drop(columns=['seconds_from_midnight'], inplace=True)

# Convert the LOB Dataset's Timestamp from seconds to a datetime format
lob_dataset['timestamp'] = base_date + pd.to_timedelta(lob_dataset['Timestamp'], unit='s')
lob_dataset.drop(columns=['Unnamed: 0', 'Timestamp'], inplace=True)

# Let's check the first few rows of both datasets after conversion
real_tape.head(), lob_dataset.head()


(   price  quantity               timestamp
 0    269         1 2025-01-02 00:00:11.067
 1    267         2 2025-01-02 00:00:11.222
 2    270         2 2025-01-02 00:00:12.338
 3    267         3 2025-01-02 00:00:13.733
 4    265         2 2025-01-02 00:00:18.321,
         Bid         Ask               timestamp
 0        []          [] 2025-01-02 00:00:00.000
 1  [[1, 6]]          [] 2025-01-02 00:00:00.279
 2  [[1, 6]]  [[800, 1]] 2025-01-02 00:00:01.333
 3  [[1, 6]]  [[799, 1]] 2025-01-02 00:00:01.581
 4  [[1, 6]]  [[798, 1]] 2025-01-02 00:00:01.643)

In [20]:
# from here on is completely wrong, a few things I need to do
# 1. merge tape into lob -> create a new csv or dataframe?
# 2. research strategy that makes the most profit (my current idea is to buy all stock that is below average and sell when is above average (but volumn can be tricky))
# 3. I can make simplify assumptions  


# Merge the Real Tape and LOB Dataset based on their timestamp columns
merged_dataset = pd.merge_asof(lob_dataset.sort_values('timestamp'), 
                               real_tape.sort_values('timestamp'), 
                               on='timestamp', 
                               direction='nearest')

# Since the Predicted Tape is already in the correct format, we'll also merge it with the merged_dataset
predicted_tape['timestamp'] = pd.to_datetime(predicted_tape['timestamp'])
merged_dataset = pd.merge_asof(merged_dataset.sort_values('timestamp'),
                               predicted_tape.sort_values('timestamp'),
                               on='timestamp',
                               direction='nearest',
                               suffixes=('', '_predicted'))

# Display the first few rows of the merged dataset to verify the merge
merged_dataset.head()


,Bid,Ask,timestamp,price,quantity,transaction_price
0,[],[],2025-01-02 00:00:00.000,269,1,267.0
1,"[[1, 6]]",[],2025-01-02 00:00:00.279,269,1,267.0
2,"[[1, 6]]","[[800, 1]]",2025-01-02 00:00:01.333,269,1,267.0
3,"[[1, 6]]","[[799, 1]]",2025-01-02 00:00:01.581,269,1,267.0
4,"[[1, 6]]","[[798, 1]]",2025-01-02 00:00:01.643,269,1,267.0


In [21]:
# Define initial simulation parameters
initial_capital = 10000  # Starting capital in USD
trade_size = 1  # Number of shares per trade
transaction_cost = 0  # Assuming zero transaction costs for simplicity

# Initialize simulation variables
capital = initial_capital
holdings = 0  # Number of shares held

# Record of trades made
trades = []

# Iterate through the merged dataset to simulate trading
for i in range(len(merged_dataset) - 1):
    current_price = merged_dataset.loc[i, 'price']
    next_price = merged_dataset.loc[i + 1, 'transaction_price']  # Using predicted price for perfect foresight
    
    # Buy if the next price is higher than the current price and we have enough capital
    if next_price > current_price and capital >= current_price:
        capital -= (current_price * trade_size + transaction_cost)
        holdings += trade_size
        trades.append(('Buy', current_price, merged_dataset.loc[i, 'timestamp']))
    
    # Sell if the next price is lower than the current price and we have holdings
    elif next_price < current_price and holdings > 0:
        capital += (current_price * trade_size - transaction_cost)
        holdings -= trade_size
        trades.append(('Sell', current_price, merged_dataset.loc[i, 'timestamp']))

# Sell any remaining holdings at the last known price
if holdings > 0:
    final_price = merged_dataset.iloc[-1]['price']
    final_timestamp = merged_dataset.iloc[-1]['timestamp']  # Corrected access to the timestamp
    capital += holdings * final_price
    trades.append(('Sell', final_price, final_timestamp))
    holdings = 0

# Recalculate the final profit or loss
profit_or_loss = capital - initial_capital

# Corrected display of the results
profit_or_loss, trades[:5]  # Display the first 5 trades to keep the output concise


(305876,
 [('Buy', 267, Timestamp('2025-01-02 00:00:11.222000')),
  ('Buy', 267, Timestamp('2025-01-02 00:00:11.253000')),
  ('Buy', 267, Timestamp('2025-01-02 00:00:11.470000')),
  ('Buy', 267, Timestamp('2025-01-02 00:00:11.532000')),
  ('Buy', 267, Timestamp('2025-01-02 00:00:11.563000'))])